This notebook contains some tests to decide on an experimental design for the simulatability experiment.

In [1]:
import krippendorff


In [2]:
from scipy.stats import ttest_ind
from scipy.stats import ttest_1samp

In [3]:

import numpy as np 
import scipy.stats as stats 
import matplotlib.pyplot as plt


In [4]:
import pandas as pd
import numpy as np

In [5]:
import random

## Hase et al. 2020.

This is a within-subject* design with 4 phases: (1) Predictions only, (2) Pre-learn test, (3) Teaching: Predictions + Explanations, (4) Eval.

Phase 1 and 3 share a set of documents as do 2 and 4.

Result: Report **average change** in user accuracy per explanation method (phase 2 vs. 4), CI and p values of mean

Additional parameters in Hase et al.:
- Balance data "by model correctness" so random guessing can't succeed: *"we ensure that true positives, false positives,
true negatives, and false negatives are equally represented in the inputs. [...] We confirm user understanding of the data
balancing in our screening test"*
- Forced choice, to not "favor overly niche explanations" (like in Ribeiro et al.)
- Separate teach and test phases
- Pre prediction phase to obtain a baseline
- **All users see the same examples**


*: One explanation method per user, some users repeat the experiment with a new dataset!?!



### Feasability

In [6]:
columns_experiment = ["user_id", "document_id", "user_label"]


In [7]:
import sqlite3
import pandas as pd
# Create your connection.
connection = sqlite3.connect("../survey/db.db")

user_df = pd.read_sql_query("SELECT * FROM users", connection)

In [8]:
import requests
import json

In [9]:
url = "http://localhost:3002"
# url = "https://survey.loris.fyi"

In [10]:
documentNr = 1
label = 0

In [11]:
user_df

,ID,access_token,current_phase,detector,explainer,document_order_a,document_order_b
0,1,OJMCOW,-1,None,None,"[11,1,6,2,9,8,0,4,7,5,3,10]","[0,10,9,7,8,3,5,2,4,1,6,11]"
1,2,TMVOTZ,-1,None,None,"[4,9,0,7,11,10,1,2,6,5,3,8]","[11,4,6,0,5,1,2,8,7,9,3,10]"
2,3,KPAFWI,-1,None,None,"[6,4,8,9,5,0,10,7,1,3,2,11]","[10,1,0,11,9,2,7,3,4,5,6,8]"
3,4,AMGHOM,-1,None,None,"[8,6,4,9,7,1,0,2,11,10,5,3]","[9,6,3,2,4,1,7,8,10,0,5,11]"
4,5,GOVHTO,-1,None,None,"[2,1,8,6,7,4,0,3,11,9,10,5]","[4,6,3,8,2,11,9,1,0,5,10,7]"
5,6,BGZHWO,-1,None,None,"[10,2,1,3,0,5,11,9,4,6,7,8]","[6,1,8,5,11,3,9,7,4,0,10,2]"
6,7,ONGAFQ,-1,None,None,"[3,11,2,9,7,8,4,1,6,5,10,0]","[8,3,4,10,6,5,0,11,1,9,7,2]"
7,8,ASQXBA,-1,None,None,"[5,3,11,1,0,6,2,10,4,9,8,7]","[2,9,1,6,3,7,11,0,4,8,10,5]"
8,9,NQIFCU,-1,None,None,"[5,8,9,0,2,1,4,7,11,3,6,10]","[0,4,2,7,6,3,10,1,5,8,11,9]"
9,10,DWQPOI,-1,None,None,"[3,5,10,2,6,1,4,11,0,8,9,7]","[2,1,4,0,9,3,7,6,8,11,10,5]"


In [12]:
df_user_study = pd.read_csv("./dataset_user_study_12.csv")

In [13]:
%%writefile run_user.py

def run_user(idx, user, url, df_user_study):
    n_learn = 16
    n_eval = 16
    n_users = 10

    mu_got_it_right_pre=0.5
    sigma_got_it_right_pre=0.05
    mu_gain = 0.2
    sigma_gain = 0.1

    def guess(detector_label,p):
        return detector_label if bool(np.random.choice([0,1],p=[1-p, p])) else not detector_label

    import requests
    import json
    import numpy as np
    import pandas as pd
    user_dist_without = lambda : np.clip(np.random.normal(mu_got_it_right_pre, sigma_got_it_right_pre, 1)[0], 0,1)
    user_dist_gain = lambda : np.clip(np.random.normal(mu_gain, sigma_gain, 1)[0], -1,1)
  
    res = requests.get(url+"/auth/"+ user["access_token"])

    print(res.text)
    auth_token = json.loads(res.text)
    headers = {'Content-Type': 'application/json','Authorization': "Bearer "+auth_token, "Content-Type": "application/json",}

    requests.post(url+"/api/submitParticipantInfo", json={
    "has_seen_explanation_methods_before": "yes",
    "has_seen_OTHERS_before": "yes",
    "level_of_expertise": "is-researcher-explainability",
    "familiarity_with_chatgpt": "occasional-use",
    "prefers_monochromatic_methods": "yes" if user["access_token"] == "DDEBUG" else "no"
    }, headers=headers)
    # go to phase 2
    requests.post(url+"/api/completeCurrentPhase", json={"expected": 0}, headers=headers)
    requests.post(url+"/api/completeCurrentPhase", json={"expected": 1}, headers=headers)
    requests.post(url+"/api/completeCurrentPhase", json={"expected": 2}, headers=headers)

    res = requests.get(url+"/api/state", headers=headers)
    state = json.loads(res.text)

    # user_df = pd.read_sql_query("SELECT * FROM users", connection) # update as group is assigned now
    # user = user_df.iloc[idx]
   # print(user[["detector", "explainer"]])
    # return state
    df_user_documents = df_user_study.loc[df_user_study.groupby("Detector").groups[state["detector"]],:].reset_index(drop=True)
    for doc_nr, row in df_user_documents.iterrows():
        p_without = user_dist_without()
        requests.post(url+"/api/submitPhase2", json={"ID": doc_nr, "label": guess(row["f(b)"], p_without)}, headers=headers)
    requests.post(url+"/api/completeCurrentPhase", json={"expected": 3}, headers=headers)

    for doc_nr, row in df_user_documents.iterrows():
        json_ = {"lickert-q{}-{}".format(question_nr, doc_nr): str(np.random.choice([1,2,3,4,5], p=[0.1,0.1,0.1,0.4,0.3])) for question_nr in range(1,6)}
        json_["document_nr"] = doc_nr
        requests.post(url+"/api/submitPhase3", json=json_, headers=headers)

    requests.post(url+"/api/completeCurrentPhase", json={"expected": 4}, headers=headers)
    for doc_nr, row in df_user_documents.iterrows():
        p_with = np.clip(p_without + user_dist_gain(), 0,1)
        requests.post(url+"/api/submitPhase4", json={"ID": doc_nr, "label": guess(row["f(b)"], p_with)}, headers=headers)
    requests.post(url+"/api/completeCurrentPhase", json={"expected": 5}, headers=headers)


Overwriting run_user.py


In [14]:
frames_list = [(idx, user, url, df_user_study) for idx, user in user_df.iterrows()]


In [15]:
from tqdm import tqdm

In [16]:
from multiprocess import Pool
from run_user import run_user



max_pool = 30

with Pool(max_pool) as p:
    pool_outputs = list(
        tqdm(
            p.starmap(run_user,
                   frames_list),
            total=len(frames_list)
        )
    )    
print(pool_outputs)


100%|██████████| 27/27 [00:00<00:00, 26976.23it/s]


[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]


In [17]:
user_df = pd.read_sql_query("SELECT * FROM users", connection) # update as group is assigned now

In [18]:
user_df.groupby(["detector", "explainer"])["ID"].count()

detector           explainer       
DetectorDetectGPT  Anchor_Explainer    3
                   LIME_Explainer      3
                   SHAP_Explainer      3
DetectorGuo        Anchor_Explainer    3
                   LIME_Explainer      3
                   SHAP_Explainer      3
DetectorRadford    Anchor_Explainer    3
                   LIME_Explainer      3
                   SHAP_Explainer      3
Name: ID, dtype: int64

In [49]:
27*6

162

In [19]:
def user_metrics(df_user_responses, df_user_study):
    detector = df_user_responses.iloc[0]["detector"]
    # explainer = df_user_responses.iloc[0]["explainer"]
    df_user_documents = df_user_study.loc[df_user_study.groupby("Detector").groups[detector],:].reset_index(drop=True)
    detector_predictions = df_user_documents["f(b)"].astype(bool)

    user_responses = df_user_responses.loc[df_user_responses.groupby("document_nr")["timestamp"].idxmax()].set_index("document_nr")["label"].astype(bool) # only keep most recent response
    # display(user_responses)
    # display(detector_predictions)
    TP = ((detector_predictions) & (user_responses)).sum()
    FP = ((~detector_predictions) & (user_responses)).sum()

    TN = ((~detector_predictions) & (~user_responses)).sum()
    FN = ((detector_predictions) & (~user_responses)).sum()

    acc = (TP+TN) / (TP+FP+TN+FN)
    # print("acc", acc)
    # print("TP", TP)
    # print("FP", FP)
    # print("TN", TN)
    # print("FN", FN)


    assert sum([TP,FP,TN,FN]) == len(detector_predictions), "Check that input is bool"
    assert (acc ==(user_responses == detector_predictions).sum() / len(detector_predictions)), "Check that input is bool: acc"

    return pd.DataFrame([(TP,TN,FP,FN, acc)], columns=["TP","TN","FP","FN", "User Accuracy"])

In [20]:
u = user_df.set_index("ID").rename_axis("user_id")[["explainer", "detector"]]


In [21]:
df_phase_2 = pd.read_sql_query("SELECT responses_phase_2.*, users.detector, users.explainer FROM responses_phase_2 INNER JOIN users ON responses_phase_2.user_id = users.ID", connection)
df_phase_4 = pd.read_sql_query("SELECT responses_phase_4.*, users.detector, users.explainer FROM responses_phase_4 INNER JOIN users ON responses_phase_4.user_id = users.ID", connection)

metrics_phase_4 = df_phase_4.groupby(["user_id"]).apply(lambda df_user_responses : user_metrics(df_user_responses,df_user_study))
metrics_phase_2 = df_phase_2.groupby(["user_id"]).apply(lambda df_user_responses : user_metrics(df_user_responses,df_user_study))


difference = metrics_phase_4 - metrics_phase_2 


In [22]:
difference.join(u).groupby(["detector"])["User Accuracy"].mean()

detector
DetectorDetectGPT    0.111111
DetectorGuo          0.083333
DetectorRadford      0.250000
Name: User Accuracy, dtype: float64

In [23]:
difference.join(u).groupby(["explainer"])["User Accuracy"].mean()

explainer
Anchor_Explainer    0.129630
LIME_Explainer      0.064815
SHAP_Explainer      0.250000
Name: User Accuracy, dtype: float64

In [24]:
difference.join(u).groupby(["detector", "explainer"])["User Accuracy"].mean()

detector           explainer       
DetectorDetectGPT  Anchor_Explainer    0.111111
                   LIME_Explainer      0.111111
                   SHAP_Explainer      0.111111
DetectorGuo        Anchor_Explainer    0.083333
                   LIME_Explainer     -0.055556
                   SHAP_Explainer      0.222222
DetectorRadford    Anchor_Explainer    0.194444
                   LIME_Explainer      0.138889
                   SHAP_Explainer      0.416667
Name: User Accuracy, dtype: float64

In [25]:
metrics_phase_4.join(u).index.get_level_values(0)

Index([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27],
      dtype='int64', name='user_id')

In [26]:
def highlight_significant(row, props=''):
  #  display(s)
    styles = [''] * len(row)
    styles[2] = 'font-weight: bold' if row["p value"] <= 0.05 else ''
    return styles

In [27]:
latex_output = []

In [28]:
def get_aggregate_results(groupby, label, caption):
    tvalues = []
    pvalues = []
    for name, group_2 in metrics_phase_2.join(u).groupby(groupby):
        group_4 =  metrics_phase_4.join(u)[np.all(metrics_phase_4.join(u)[groupby].values == name, axis=1)]
        tvalue, pvalue = ttest_ind(group_2["User Accuracy"],group_4["User Accuracy"])
        tvalues.append(tvalue)
        pvalues.append(pvalue)

    df_aggregate_results = pd.DataFrame(difference.join(u).groupby(groupby)["User Accuracy"].mean())

    df_aggregate_results = df_aggregate_results.join(pd.DataFrame(metrics_phase_2.join(u).groupby(groupby)["User Accuracy"].mean()).rename({"User Accuracy":"Before"}, axis=1))
    df_aggregate_results = df_aggregate_results.join(pd.DataFrame(metrics_phase_4.join(u).groupby(groupby)["User Accuracy"].mean()).rename({"User Accuracy":"After"}, axis=1))

    df_aggregate_results["t value"] = tvalues
    df_aggregate_results["p value"] = pvalues

    df_aggregate_results.rename(columns={"User Accuracy":"Increase in User Accuracy"}, inplace=True)
    df_aggregate_results = df_aggregate_results.reindex(sorted(df_aggregate_results.columns), axis=1)
    result = df_aggregate_results.style.apply(highlight_significant, axis=1)\
        .map_index(lambda v: "rotatebox:{45}--rwrap;", level=0, axis=1).format(precision=2).hide(["t value"], axis=1).format_index(escape="latex", axis=0)
    latex_output.append(result.to_latex(environment="longtable", 
                                        convert_css=True, 
                                        clines="all;data", 
                                        hrules=True, 
                                        caption=caption, 
                                        label=label))
    return result

In [29]:
get_aggregate_results(["detector"], "resultsuserstudydetector", "Results aggregated by detector")

,After,Before,Increase in User Accuracy,p value
detector,,,,
DetectorDetectGPT,0.68,0.56,0.11,0.04
DetectorGuo,0.63,0.55,0.08,0.28
DetectorRadford,0.74,0.49,0.25,0.00


In [30]:
get_aggregate_results(["explainer"], "resultsuserstudyexplainer", "Results aggregated by explainer")

,After,Before,Increase in User Accuracy,p value
explainer,,,,
Anchor\_Explainer,0.64,0.51,0.13,0.01
LIME\_Explainer,0.68,0.61,0.06,0.39
SHAP\_Explainer,0.73,0.48,0.25,0.00


In [31]:
get_aggregate_results(["explainer", "detector"], "resultsuserstudyexplainerdetector", "Results aggregated by explainer and detector pairing")

## Lickert Items

In [32]:
df_phase_3 = pd.read_sql_query("SELECT responses_phase_3.*, users.detector, users.explainer FROM responses_phase_3 INNER JOIN users ON responses_phase_3.user_id = users.ID", connection)
df_phase_3

,ID,timestamp,label,user_id,document_nr,question_nr,detector,explainer
0,1,2024-02-26 09:11:20,4,3,0,1,DetectorDetectGPT,SHAP_Explainer
1,2,2024-02-26 09:11:20,5,3,0,5,DetectorDetectGPT,SHAP_Explainer
2,3,2024-02-26 09:11:20,4,3,0,4,DetectorDetectGPT,SHAP_Explainer
3,4,2024-02-26 09:11:20,2,3,0,2,DetectorDetectGPT,SHAP_Explainer
4,5,2024-02-26 09:11:20,1,6,0,3,DetectorRadford,SHAP_Explainer
...,...,...,...,...,...,...,...,...
1615,1616,2024-02-26 09:11:34,4,25,11,2,DetectorGuo,SHAP_Explainer
1616,1617,2024-02-26 09:11:34,5,7,11,1,DetectorDetectGPT,Anchor_Explainer
1617,1618,2024-02-26 09:11:34,4,22,11,4,DetectorDetectGPT,LIME_Explainer
1618,1619,2024-02-26 09:11:34,4,22,11,2,DetectorDetectGPT,LIME_Explainer


In [33]:
user_responses = df_phase_3.loc[df_phase_3.groupby(["user_id", "question_nr", "document_nr"])["timestamp"].idxmax()].set_index(["user_id", "document_nr", "question_nr"]).drop(["timestamp", "ID"], axis=1)
user_responses

label           detector         explainer
user_id document_nr question_nr                                            
1       0           1                5  DetectorDetectGPT    LIME_Explainer
        1           1                4  DetectorDetectGPT    LIME_Explainer
        2           1                5  DetectorDetectGPT    LIME_Explainer
        3           1                5  DetectorDetectGPT    LIME_Explainer
        4           1                2  DetectorDetectGPT    LIME_Explainer
...                                ...                ...               ...
27      7           5                3        DetectorGuo  Anchor_Explainer
        8           5                4        DetectorGuo  Anchor_Explainer
        9           5                4        DetectorGuo  Anchor_Explainer
        10          5                1        DetectorGuo  Anchor_Explainer
        11          5                2        DetectorGuo  Anchor_Explainer

[1620 rows x 3 columns]

In [34]:
user_responses.groupby(["detector", "explainer", "question_nr"]).mean()

label
detector          explainer        question_nr          
DetectorDetectGPT Anchor_Explainer 1            3.750000
                                   2            3.888889
                                   3            3.861111
                                   4            3.444444
                                   5            3.638889
                  LIME_Explainer   1            3.861111
                                   2            3.388889
                                   3            3.638889
                                   4            3.500000
                                   5            3.583333
                  SHAP_Explainer   1            3.416667
                                   2            3.583333
                                   3            3.888889
                                   4            3.722222
                                   5            3.666667
DetectorGuo       Anchor_Explainer 1            3.805556
                                   2            3.666667
                                   3            3.666667
                                   4            3.916667
                                   5            3.527778
                  LIME_Explainer   1            3.361111
                                   2            3.750000
                                   3            3.638889
                                   4            3.444444
                                   5            3.750000
                  SHAP_Explainer   1            3.472222
                                   2            3.694444
                                   3            4.138889
                                   4            3.694444
                                   5            3.444444
DetectorRadford   Anchor_Explainer 1            3.805556
                                   2            3.416667
                                   3            3.833333
                                   4            3.527778
                                   5            3.722222
                  LIME_Explainer   1            3.888889
                                   2            3.333333
                                   3            3.916667
                                   4            3.611111
                                   5            3.638889
                  SHAP_Explainer   1            3.666667
                                   2            3.472222
                                   3            3.611111
                                   4            3.861111
                                   5            3.666667

In [35]:
groupby = [ "explainer", "document_nr"]
alpha_values = []
for name, group in user_responses.groupby(groupby):
    cannonical_form = group.reset_index().pivot(columns=["question_nr"], values=["label"], index=["user_id", "document_nr"])
    # print(cannonical_form)
    # print()
    alpha_values.append((*name, krippendorff.alpha(cannonical_form, level_of_measurement="ordinal")))
df_krippendorff_alpha = pd.DataFrame(alpha_values, columns=groupby + ["alpha"])
df_krippendorff_alpha.groupby(groupby[0:-1]).describe()["alpha"]


,count,mean,std,min,25%,50%,75%,max
explainer,,,,,,,,
Anchor_Explainer,12.0,0.006072,0.106748,-0.077428,-0.043318,-0.026709,0.027414,0.320709
LIME_Explainer,12.0,-0.007458,0.044840,-0.093244,-0.031189,-0.010654,0.017700,0.079639
SHAP_Explainer,12.0,-0.000674,0.084859,-0.087306,-0.061781,-0.026373,0.039016,0.180106


In [36]:
def highlight_significant(row, props=''):
  #  display(s)
    styles = [''] * len(row)
    styles[0] = 'font-weight: bold' if row["p value"] <= 0.05 else ''
    return styles

In [37]:
def get_aggregate_results_lickert(groupby, label, caption):
    tvalues = []
    pvalues = []
    for name, group in user_responses.groupby(groupby):
        tvalue, pvalue = ttest_1samp(group["label"], popmean=3)
        tvalues.append(tvalue)
        pvalues.append(pvalue)
        
        

    df_aggregate_results = pd.DataFrame(user_responses.groupby(groupby)["label"].mean())


    df_aggregate_results["t value"] = tvalues
    df_aggregate_results["p value"] = pvalues

    df_aggregate_results = df_aggregate_results.reindex(sorted(df_aggregate_results.columns), axis=1)
    result = df_aggregate_results.style.apply(highlight_significant, axis=1)\
        .map_index(lambda v: "rotatebox:{45}--rwrap;", level=0, axis=1).format(precision=2).hide(["t value"], axis=1).format_index(escape="latex", axis=0)
    latex_output.append(result.to_latex(environment="longtable", 
                                        convert_css=True, 
                                        clines="all;data", 
                                        hrules=True, 
                                        caption=caption, 
                                        label=label))

    
    return result


In [38]:
get_aggregate_results_lickert(["detector", "explainer", "question_nr"], "lickert-detector-explainer","Lickert Scale Items on detector-explainer level")

In [39]:
get_aggregate_results_lickert([ "explainer", "question_nr"], "lickert-explainer","Lickert Scale Items on explainer level")

In [40]:
for  l in latex_output:
    print(l)

\begin{longtable}{lrrrr}
\caption{Results aggregated by detector} \label{resultsuserstudydetector} \\
\toprule
 & \rotatebox{45}{After} & \rotatebox{45}{Before} & \rotatebox{45}{Increase in User Accuracy} & \rotatebox{45}{p value} \\
detector &  &  &  &  \\
\midrule
\endfirsthead
\caption[]{Results aggregated by detector} \\
\toprule
 & \rotatebox{45}{After} & \rotatebox{45}{Before} & \rotatebox{45}{Increase in User Accuracy} & \rotatebox{45}{p value} \\
detector &  &  &  &  \\
\midrule
\endhead
\midrule
\multicolumn{5}{r}{Continued on next page} \\
\midrule
\endfoot
\bottomrule
\endlastfoot
DetectorDetectGPT & 0.68 & 0.56 & \bfseries 0.11 & 0.04 \\
\cline{1-5}
DetectorGuo & 0.63 & 0.55 & 0.08 & 0.28 \\
\cline{1-5}
DetectorRadford & 0.74 & 0.49 & \bfseries 0.25 & 0.00 \\
\cline{1-5}
\end{longtable}

\begin{longtable}{lrrrr}
\caption{Results aggregated by explainer} \label{resultsuserstudyexplainer} \\
\toprule
 & \rotatebox{45}{After} & \rotatebox{45}{Before} & \rotatebox{45}{Increase 

In [41]:




# # "Reproducing" information from Table 1/2 in Hase et al.:


# user_metrics_eval = df_phase_4.groupby(["user_id"]).apply(lambda x : user_metrics(x,df_user_study)) # TODO
# df_change = user_metrics_eval - user_metrics_pre
# df_change = df_change.rename(columns={"User Accuracy": "Change in User Accuracy"})

# user_acc_col = df_change["Change in User Accuracy"] # for convenience

# # use student t for low number of samples
# lower, upper = stats.t.interval(
# confidence=0.95, 
# df=len(user_acc_col)-1, # degrees of freedom = # samples - 1 for mean
#             loc=user_acc_col.mean(), 
#             scale=stats.sem(user_acc_col)
#             ) 

# p_val = ttest_ind(user_metrics_eval["User Accuracy"],user_metrics_pre["User Accuracy"]).pvalue

# k_alpha = krippendorff.alpha(reliability_data=df_phase_4.groupby(["user_id"]).apply(lambda df : user_responses.astype(int).to_list()).to_list())



# lower_b, upper_b = stats.bootstrap((user_acc_col,), np.mean, confidence_level=0.95,).confidence_interval

# # print results
# #  print("Mean change in acc",user_acc_col.mean())
# # print("CI for mean change: [{},{}]".format(lower,upper))

# ##    print("CI by bootstrap: [{},{}]".format(lower_b, upper_b))


# #  print("p=%.10f" % p_val, "significant (< 0.05)" if p_val < 0.05 else "NOT significant (> 0.05)")
# #   print("Krippendorff between users: {}".format(k_alpha))
# return p_val

In [42]:
# def evaluate_user_study(df_user_study, df_phase_2, df_phase_4):
#     # "Reproducing" information from Table 1/2 in Hase et al.:
#     user_metrics_pre = df_phase_2.groupby(["user_id"]).apply(lambda x : user_metrics(x,df_user_study))

#     user_metrics_eval = df_phase_4.groupby(["user_id"]).apply(lambda x : user_metrics(x,df_user_study)) # TODO
#     df_change = user_metrics_eval - user_metrics_pre
#     df_change = df_change.rename(columns={"User Accuracy": "Change in User Accuracy"})

#     user_acc_col = df_change["Change in User Accuracy"] # for convenience

#     # use student t for low number of samples
#     lower, upper = stats.t.interval(
#     confidence=0.95, 
#     df=len(user_acc_col)-1, # degrees of freedom = # samples - 1 for mean
#               loc=user_acc_col.mean(), 
#               scale=stats.sem(user_acc_col)
#               ) 
    
#     p_val = ttest_ind(user_metrics_eval["User Accuracy"],user_metrics_pre["User Accuracy"]).pvalue

#     k_alpha = krippendorff.alpha(reliability_data=df_phase_4.groupby(["user_id"]).apply(lambda df : user_responses.astype(int).to_list()).to_list())



#     lower_b, upper_b = stats.bootstrap((user_acc_col,), np.mean, confidence_level=0.95,).confidence_interval
    
#     # print results
#   #  print("Mean change in acc",user_acc_col.mean())
#    # print("CI for mean change: [{},{}]".format(lower,upper))

# ##    print("CI by bootstrap: [{},{}]".format(lower_b, upper_b))

    
#   #  print("p=%.10f" % p_val, "significant (< 0.05)" if p_val < 0.05 else "NOT significant (> 0.05)")
#  #   print("Krippendorff between users: {}".format(k_alpha))
#     return p_val
    

In [43]:
# def simulate_hase(

#         n_learn = 16,
#         n_eval = 16,
#         n_users = 10,

#         mu_got_it_right_pre=0.5,
#         sigma_got_it_right_pre=0.05,

#         mu_gain = 0.1,
#         sigma_gain = 0.1,


# ):
#     users = []
#     user_dist_without = lambda : np.clip(np.random.normal(mu_got_it_right_pre, sigma_got_it_right_pre, 1)[0], 0,1)
#     user_dist_gain = lambda : np.clip(np.random.normal(mu_gain, sigma_gain, 1)[0], -1,1)
#     for i in range(1, n_users+1):
#         p_without = user_dist_without()
#         p_with = np.clip(p_without + user_dist_gain(), 0,1)
#         users.append(("u_%s" % i, p_without ,p_with))
#     documents_learn_1_2 = ["l_%s" % i for i in range(1,n_learn+1)]
#     documents_pre_eval = ["e_%s" % i for i in range(1,n_eval+1)]

#     df_detector_output = mock_detector_responses(documents_pre_eval)

#     responses_pre, responses_eval = mock_user_responses(df_detector_output, documents_pre_eval, users)
#     df_pre =pd.DataFrame(responses_pre, columns=columns_experiment)
#     df_eval =pd.DataFrame(responses_eval, columns=columns_experiment)

#  #   print("# responses pre", len(responses_pre))
#   #  print("# responses pre per method", len(responses_pre)/3)
#    # print("Each user saw {} instances. ".format(2*n_learn + 2*n_eval) )
#    # print("Used {} unique documents. A set of {} in phase 1 and 3; and a set of {} in phase 2 and 4.".format(n_learn + n_eval,n_learn, n_eval))

# #    print("Results based on {} unique eval documents.".format(n_eval))

# #    print("Results based on {} datapoints.".format(len(responses_eval)))
#     p_value = evaluate_user_study(df_detector_output, df_pre, df_eval)
#     return p_value